<a href="https://colab.research.google.com/github/ysfArslan1/tf-idf/blob/main/tfidf_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import os

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words =  set(stopwords.words('english'))
stemmer = nltk.porter.PorterStemmer()

In [ ]:
def review_to_wordlist(review, remove_stopwords=True):
    # Clean the text, with the option to remove stopwords.
    
    # Convert words to lower case and split them
    words = review.lower().split()

    # Optionally remove stop words (true by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    review_text = " ".join(words)

    # Clean the text
    review_text = re.sub(r"[^A-Za-z0-9(),!.?\'\`]", " ", review_text)
    review_text = re.sub(r"\'s", " 's ", review_text)
    review_text = re.sub(r"\'ve", " 've ", review_text)
    review_text = re.sub(r"n\'t", " 't ", review_text)
    review_text = re.sub(r"\'re", " 're ", review_text)
    review_text = re.sub(r"\'d", " 'd ", review_text)
    review_text = re.sub(r"\'ll", " 'll ", review_text)
    review_text = re.sub(r",", " ", review_text)
    review_text = re.sub(r"\.", " ", review_text)
    review_text = re.sub(r"!", " ", review_text)
    review_text = re.sub(r"\(", " ( ", review_text)
    review_text = re.sub(r"\)", " ) ", review_text)
    review_text = re.sub(r"\?", " ", review_text)
    review_text = re.sub(r"\s{2,}", " ", review_text)
    
    words = review_text.split()
    
    # Shorten words to their stems
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in words]
    
    review_text = " ".join(stemmed_words)
    
    # Return a list of words
    return(review_text)

In [ ]:
def make_data(path):
  veriler = []
  pdfler = os.listdir(path)
  for pdf in pdfler:
    try:
        veri= {
            "isim":"",
        }
        veri.update({"isim" : pdf})
        dosya = open(path+pdf,"r")
        for metin in dosya:
            veri.update({"metin":metin})
        
        dosya.close()
        veriler.append(veri)
    except:
        print( "hata")
  return veriler

In [ ]:
path_groundtruth = "/content/drive/MyDrive/kodlamalar/sevinc_hoca/nlp/Dataset/groundturth.txt"
dosya = open(path_groundtruth,"r")
sonuc = ""
for metin in dosya:
  sonuc = metin
dosya.close()
groundtruth = metin

In [ ]:
path_manu= "/content/drive/MyDrive/kodlamalar/sevinc_hoca/nlp/Dataset/manuscripts/"
path_revi= "/content/drive/MyDrive/kodlamalar/sevinc_hoca/nlp/Dataset/reviewers/"

manuscripts_veriler = make_data(path_manu)
revievers_veriler = make_data(path_revi)


In [ ]:
metin = manuscripts_veriler[0]["metin"]
print(metin)
metin = review_to_wordlist(metin,True)

massless and massive higher spins from anti-de sitter space waveguide higgs mechanism to massive higher-spin gauge fields is an outstanding openproblem. we investigate this issue in the context of kaluza-kleincompactification. starting from a free massless higher-spin field in$(d+2)$-dimensional anti-de sitter space and compactifying over a finiteangular wedge, we obtain an infinite tower of heavy, light and masslesshigher-spin fields in $(d+1)$-dimensional anti-de sitter space. all massivehigher-spin fields are described gauge invariantly in terms of st\"ueckelbergfields. the spectrum depends on the boundary conditions imposed at both ends ofthe wedges. we obseved that higher-derivative boundary condition is inevitablefor spin greater than three. for some higher-derivative boundary conditions,equivalently, spectrum-dependent boundary conditions, we get a non-unitaryrepresentation of partially-massless higher-spin fields of varying depth. wepresent intuitive picture which higher-deriva

In [ ]:
print(metin)

massless massiv higher spin anti de sitter space waveguid higg mechan massiv higher spin gaug field outstand openproblem investig issu context kaluza kleincompactif start free massless higher spin field in ( d 2 ) dimension anti de sitter space compactifi finiteangular wedg obtain infinit tower heavi light masslesshigh spin field ( d 1 ) dimension anti de sitter space massivehigh spin field describ gaug invari term st ueckelbergfield spectrum depend boundari condit impos end ofth wedg obsev higher deriv boundari condit inevitablefor spin greater three higher deriv boundari condit equival spectrum depend boundari condit get non unitaryrepresent partial massless higher spin field vari depth wepres intuit pictur higher deriv boundari condit yieldnon unitari system term boundari action argu isotropiclifshitz interfac o ( n ) heisenberg magnet o ( n ) gross neveu modelprovid holograph dual conform field theori propos experimentaltest ( invers ) higg mechan massiv partial masslesshigh spin f

In [ ]:

for i in range(len(manuscripts_veriler)):
  metin = manuscripts_veriler[i]["metin"]
  metin = review_to_wordlist(metin,True)
  manuscripts_veriler[i].update({"metin":metin})

In [ ]:
for i in range(len(revievers_veriler)):
  metin = revievers_veriler[i]["metin"]
  metin = review_to_wordlist(metin,True)
  revievers_veriler[i].update({"metin":metin})

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
def cosine_sim(text1, reviwers):
    score = []
    for i in range(len(reviwers)):
      dic = {
          "isim":"",
          "result":0
      }
      text2 = reviwers[i]["metin"]
      tfidf = vectorizer.fit_transform([text1, text2])
      deger = ((tfidf * tfidf.T).A)[0,1]
      dic.update({"isim":reviwers[i]["isim"]})
      dic.update({"result":deger})
      score.append(dic)
    return score

In [ ]:
len(manuscripts_veriler)

705

In [ ]:
Tfidf_scores_deneme = []
for i in range(10):
    veri={
        "isim":"",
        "karsilastirma_sonucları":[]
    }
    score = cosine_sim(manuscripts_veriler[i]["metin"], revievers_veriler)
    print(i , " ", manuscripts_veriler[i]["isim"])
    veri.update({"isim":manuscripts_veriler[i]["isim"]})
    veri.update({"karsilastirma_sonucları":score})
    Tfidf_scores_deneme.append(veri)

0   1605.06526.txt
1   1607.05360.txt
2   1403.7583.txt
3   1511.07071.txt
4   1603.06997.txt
5   1604.07571.txt
6   1402.5070.txt
7   1605.08502.txt
8   1508.01016.txt
9   1605.02590 (1).txt


In [ ]:
makale = Tfidf_scores_deneme[0]["isim"]
karsilastirma_sonucları = Tfidf_scores_deneme[0]["karsilastirma_sonucları"]
db = pd.DataFrame(karsilastirma_sonucları)

In [ ]:
#help(db.sort_values)

In [ ]:
db = db.sort_values('result',ascending=False ,ignore_index = True )

In [ ]:
print(makale)
print(db.head(10))

1605.06526.txt
                   isim    result
0         wu, m. w..txt  0.207700
1     kang, hyesung.txt  0.166137
2  wolfe, arthur m..txt  0.120323
3       ryu, dongsu.txt  0.119028
4      jones, t. w..txt  0.119028
5       frank, adam.txt  0.119028
6      beck, rainer.txt  0.115232
7      ekers, r. d..txt  0.109985
8      dolag, klaus.txt  0.107582
9      matsuura, m..txt  0.105843


In [ ]:
dizi = groundtruth.split("'")
print("1++++++++++++++++++",dizi[0])
print("2++++++++++++++++++",dizi[1])

1++++++++++++++++++ gr-qc;0701012.txt	rezzolla, luciano	boyanovsky, d.	sola, joan	kalogera, v.	hu, b. l.	melia, fulvio	szydlowski, marek	cumming, a.	amelino-camelia, giovanni	souradeep, tarun	saharian, a. a.	shibata, masaru	tsujikawa, shinji	berti, emanuele	yunes, nicolas	shapiro, stuart l.	sathyaprakash, b. s.	li, baojiu	melatos, a.	kumar, p.	jones, d. i.	vecchio, a.	allen, b.	o
2++++++++++++++++++ shaughnessy, r.	siemens, x.



'gr-qc;0701012.txt\trezzolla, luciano\tboyanovsky, d.\tsola, joan\tkalogera, v.\thu, b. l.\tmelia, fulvio\tszydlowski, marek\tcumming, a.\tamelino-camelia, giovanni\tsouradeep, tarun\tsaharian, a. a.\tshibata, masaru\ttsujikawa, shinji\tberti, emanuele\tyunes, nicolas\tshapiro, stuart l.\tsathyaprakash, b. s.\tli, baojiu\tmelatos, a.\tkumar, p.\tjones, d. i.\tvecchio, a.\tallen, b.\to'

In [ ]:
Tfidf_scores = []
for i in range(len(manuscripts_veriler)):
    veri={
        "isim":"",
        "karsilastirma_sonucları":[]
    }
    score = cosine_sim(manuscripts_veriler[i]["metin"], revievers_veriler)
    print(i , " ", manuscripts_veriler[i]["isim"])
    veri.update({"isim":manuscripts_veriler[i]["isim"]})
    veri.update({"karsilastirma_sonucları":score})
    Tfidf_scores.append(veri)
    

0   1605.06526.txt
1   1607.05360.txt
2   1403.7583.txt
3   1511.07071.txt
4   1603.06997.txt
5   1604.07571.txt
6   1402.5070.txt
7   1605.08502.txt
8   1508.01016.txt
9   1605.02590 (1).txt
10   1605.03906.txt
11   1601.00320.txt
12   1606.00621 (1).txt
13   1606.00621.txt
14   1412.5028.txt
15   1607.05703.txt
16   1604.01267.txt
17   1201.3343.txt
18   1603.02676.txt
19   1602.03237.txt
20   1601.05386.txt


KeyboardInterrupt: ignored